In [1]:
import boto3
import pandas as pd
from io import StringIO

# Define your bucket name and file key (file path in S3)
BUCKET_NAME = "blue-blood-data"
FILE_KEY = "BigQuery Blue Blood DB Data.csv"  # Change to your actual file path in S3

# Create an S3 client
s3 = boto3.client("s3")


In [2]:

# Fetch the file from S3
response = s3.get_object(Bucket=BUCKET_NAME, Key=FILE_KEY)

# Read the CSV file into a pandas DataFrame
csv_content = response["Body"].read().decode("utf-8")
df = pd.read_csv(StringIO(csv_content))

# Print DataFrame
print(df.head())

   subject_id   prescription_start     prescription_drug  \
0       10013  2125-10-05T00:00:00     Phenylephrine HCl   
1       10013  2125-10-05T00:00:00           Vasopressin   
2       10013  2125-10-05T00:00:00  Iso-Osmotic Dextrose   
3       10013  2125-10-05T00:00:00            Dobutamine   
4       10013  2125-10-05T00:00:00               Aspirin   

  prescription_dose_val_rx prescription_dose_unit_rx        pre_charttime  \
0                       60                        mg  2125-10-04T23:59:00   
1                      100                      UNIT  2125-10-04T23:59:00   
2                       50                        ml  2125-10-04T23:59:00   
3                      250                        mg  2125-10-04T23:59:00   
4                      325                        mg  2125-10-04T23:59:00   

   pre_ph  pre_pco2  pre_po2  pre_bicarbonate  ...  post_fio2_chartevents  \
0     7.3      63.0     60.0              NaN  ...              60.000002   
1     7.3      63.0   

In [3]:
df

,subject_id,prescription_start,prescription_drug,prescription_dose_val_rx,prescription_dose_unit_rx,pre_charttime,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,10013,2125-10-05T00:00:00,Phenylephrine HCl,60,mg,2125-10-04T23:59:00,7.30,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10013,2125-10-05T00:00:00,Vasopressin,100,UNIT,2125-10-04T23:59:00,7.30,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10013,2125-10-05T00:00:00,Iso-Osmotic Dextrose,50,ml,2125-10-04T23:59:00,7.30,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10013,2125-10-05T00:00:00,Dobutamine,250,mg,2125-10-04T23:59:00,7.30,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10013,2125-10-05T00:00:00,Aspirin,325,mg,2125-10-04T23:59:00,7.30,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,44212,2123-11-25T00:00:00,Vancomycin,500,mg,2123-11-24T21:56:00,7.24,48.0,76.0,NaN,...,80.000000,343.900000,198.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2178,44212,2123-11-25T00:00:00,0.9% Sodium Chloride,100,mL,2123-11-24T21:56:00,7.24,48.0,76.0,NaN,...,80.000000,343.900000,198.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2179,44212,2123-11-25T00:00:00,Piperacillin-Tazobactam Na,2.25,g,2123-11-24T21:56:00,7.24,48.0,76.0,NaN,...,80.000000,343.900000,198.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2180,44212,2123-11-25T00:00:00,Prismasate (B22 K4),5000,mL,2123-11-24T21:56:00,7.24,48.0,76.0,NaN,...,80.000000,343.900000,198.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:

import pandas as pd
import numpy as np

# Print values of columns that start with 'pre_' or 'post_'
'''for col in df.columns:
    if col.startswith("pre_") or col.startswith("post_"):
        print(f"Column: {col}")
        print(df[col])'''
for col in df.select_dtypes(include=[np.number]).columns:
    if (df[col] < 0).any():
        print(f"Column '{col}' contains negative values.", df[col].values.min())


Column 'pre_baseexcess' contains negative values. -16.0
Column 'post_baseexcess' contains negative values. -17.0


In [5]:
import pandas as pd
import numpy as np


# Identify columns starting with "pre_" or "post_"
pre_post_columns = [col for col in df.columns if col.startswith("pre_") or col.startswith("post_")]

# Create a new DataFrame with only the selected columns
blood_df = df[pre_post_columns].copy()

# Replace NaN values with -200
blood_df.fillna(-200, inplace=True)

blood_df = blood_df.drop('pre_charttime', axis=1)
blood_df = blood_df.drop('post_charttime', axis=1)





# Optionally, save the new dataset
# blood_df.to_csv("blood_data.csv", index=False)


In [6]:
blood_df


,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,pre_baseexcess,pre_totalco2,pre_hematocrit,pre_hemoglobin,pre_sodium,pre_potassium,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,7.30,63.0,60.0,-200.0,2.0,32.0,-200.0,-200.0,-200.0,-200.0,...,60.000002,231.550017,191.666659,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
1,7.30,63.0,60.0,-200.0,2.0,32.0,-200.0,-200.0,-200.0,-200.0,...,60.000002,231.550017,191.666659,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
2,7.30,63.0,60.0,-200.0,2.0,32.0,-200.0,-200.0,-200.0,-200.0,...,60.000002,231.550017,191.666659,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
3,7.30,63.0,60.0,-200.0,2.0,32.0,-200.0,-200.0,-200.0,-200.0,...,60.000002,231.550017,191.666659,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
4,7.30,63.0,60.0,-200.0,2.0,32.0,-200.0,-200.0,-200.0,-200.0,...,60.000002,231.550017,191.666659,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,7.24,48.0,76.0,-200.0,-6.0,22.0,-200.0,-200.0,-200.0,-200.0,...,80.000000,343.900000,198.750000,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
2178,7.24,48.0,76.0,-200.0,-6.0,22.0,-200.0,-200.0,-200.0,-200.0,...,80.000000,343.900000,198.750000,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
2179,7.24,48.0,76.0,-200.0,-6.0,22.0,-200.0,-200.0,-200.0,-200.0,...,80.000000,343.900000,198.750000,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
2180,7.24,48.0,76.0,-200.0,-6.0,22.0,-200.0,-200.0,-200.0,-200.0,...,80.000000,343.900000,198.750000,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0


In [7]:
#
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()

ndArray = scaler.fit_transform(blood_df)


##Scaled Dataframe


#Final DataFrame Dimensions
281 rows x 50 columns 

#Variables to Remember
df_scaled - scaled dataframe
blood_df - dataframe with null values (-200) pre-scaling



In [8]:
df_scaled = pd.DataFrame(ndArray, columns = blood_df.columns)

df_scaled

,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,pre_baseexcess,pre_totalco2,pre_hematocrit,pre_hemoglobin,pre_sodium,pre_potassium,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,0.0,0.0,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,0.0,0.0,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,0.0,0.0,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,0.0,0.0,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,0.0,0.0,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,0.0,0.0,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2178,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,0.0,0.0,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2179,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,0.0,0.0,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2180,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,0.0,0.0,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
import train_ctgan

In [10]:
!pip install sagemaker boto3 sdv



In [11]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
import os

boto_session = boto3.Session(region_name='us-east-1')
sagemaker_session = sagemaker.Session(boto_session=boto_session)

role = "arn:aws:iam::211125439249:role/service-role/AmazonSageMaker-ExecutionRole-20250314T153928"

/Users/neha/opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[04/10/25 18:49:03] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=361608;file:///Users/neha/opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=455296;file:///Users/neha/opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/neha/Library/Application Support/sagemaker/config.yaml


[04/10/25 18:49:04] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=240365;file:///Users/neha/opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=287723;file:///Users/neha/opt/anaconda3/envs/bb_dev/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

In [12]:
import pandas as pd
df = pd.read_csv('final_df.csv')

df

,subject_id,prescription_start,prescription_rx_embeddings,prescription_dose_val_rx,prescription_dose_unit_rx,pre_charttime,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,10013,2125-10-05T00:00:00,[ 3.5185558e-01 1.2351961e-01 -1.2304356e-01 ...,0.010317,0,2125-10-04T23:59:00,0.500,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10013,2125-10-05T00:00:00,[ 0.45182744 0.3218944 -0.5210766 0.315588...,0.011905,12,2125-10-04T23:59:00,0.500,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10013,2125-10-05T00:00:00,[ 4.5976555e-01 1.9232908e-01 -5.7382131e-01 ...,0.009921,11,2125-10-04T23:59:00,0.500,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10013,2125-10-05T00:00:00,[ 6.26637757e-01 2.61670560e-01 -2.40684357e-...,0.017857,0,2125-10-04T23:59:00,0.500,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10013,2125-10-05T00:00:00,[ 4.15423244e-01 -1.28793076e-01 -2.01883331e-...,0.020833,0,2125-10-04T23:59:00,0.500,0.741935,0.023018,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,44212,2123-11-25T00:00:00,[ 0.35067722 0.1657952 -0.11939768 0.529637...,0.027778,0,2123-11-24T21:56:00,0.375,0.500000,0.063939,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2178,44212,2123-11-25T00:00:00,[-1.67207658e+00 6.26857281e-02 -3.97092104e-...,0.011905,11,2123-11-24T21:56:00,0.375,0.500000,0.063939,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2179,44212,2123-11-25T00:00:00,[ 3.46226931e-01 1.31596521e-01 -3.90895218e-...,0.008026,2,2123-11-24T21:56:00,0.375,0.500000,0.063939,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2180,44212,2123-11-25T00:00:00,[ 0.44693142 0.00798898 -0.14514387 1.489820...,0.206349,11,2123-11-24T21:56:00,0.375,0.500000,0.063939,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df = df.drop(columns = ['subject_id', 'pre_charttime', 'post_charttime', 'prescription_start'], axis = 1)

df

,prescription_rx_embeddings,prescription_dose_val_rx,prescription_dose_unit_rx,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,pre_baseexcess,pre_totalco2,pre_hematocrit,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,[ 3.5185558e-01 1.2351961e-01 -1.2304356e-01 ...,0.010317,0,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,[ 0.45182744 0.3218944 -0.5210766 0.315588...,0.011905,12,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,[ 4.5976555e-01 1.9232908e-01 -5.7382131e-01 ...,0.009921,11,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,[ 6.26637757e-01 2.61670560e-01 -2.40684357e-...,0.017857,0,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,[ 4.15423244e-01 -1.28793076e-01 -2.01883331e-...,0.020833,0,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,[ 0.35067722 0.1657952 -0.11939768 0.529637...,0.027778,0,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2178,[-1.67207658e+00 6.26857281e-02 -3.97092104e-...,0.011905,11,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2179,[ 3.46226931e-01 1.31596521e-01 -3.90895218e-...,0.008026,2,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2180,[ 0.44693142 0.00798898 -0.14514387 1.489820...,0.206349,11,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
import pandas as pd
import numpy as np
import re

def clean_and_convert(x):
    if pd.notna(x):
        try:
            # Remove square brackets if they exist
            x = re.sub(r'[\[\]]', '', x)
            # Replace multiple spaces with a single space
            cleaned = re.sub(r'\s+', ' ', x.strip())
            # Convert to numpy array of floats
            return np.array([float(i) for i in cleaned.split(' ')])
        except Exception as e:
            # If there's an error during conversion, print the error and return the original value
            print(f"Error processing value: {x}. Error: {e}")
            return x  # Return the original value in case of error
    return x  # If NaN, return as is

In [15]:
# Check the values before applying the function
print(df['prescription_rx_embeddings'].head())

0    [ 3.5185558e-01  1.2351961e-01 -1.2304356e-01 ...
1    [ 0.45182744  0.3218944  -0.5210766   0.315588...
2    [ 4.5976555e-01  1.9232908e-01 -5.7382131e-01 ...
3    [ 6.26637757e-01  2.61670560e-01 -2.40684357e-...
4    [ 4.15423244e-01 -1.28793076e-01 -2.01883331e-...
Name: prescription_rx_embeddings, dtype: object


In [16]:
# Apply the function to the 'prescription_rx_embeddings' column
df['prescription_rx_embeddings'] = df['prescription_rx_embeddings'].apply(clean_and_convert)

# Check the result
print(df['prescription_rx_embeddings'].head())


0    [0.35185558, 0.12351961, -0.12304356, -0.33200...
1    [0.45182744, 0.3218944, -0.5210766, 0.3155888,...
2    [0.45976555, 0.19232908, -0.57382131, 1.349748...
3    [0.626637757, 0.26167056, -0.0240684357, 0.163...
4    [0.415423244, -0.128793076, -0.201883331, 0.05...
Name: prescription_rx_embeddings, dtype: object


In [17]:
df

,prescription_rx_embeddings,prescription_dose_val_rx,prescription_dose_unit_rx,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,pre_baseexcess,pre_totalco2,pre_hematocrit,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,"[0.35185558, 0.12351961, -0.12304356, -0.33200...",0.010317,0,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"[0.45182744, 0.3218944, -0.5210766, 0.3155888,...",0.011905,12,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"[0.45976555, 0.19232908, -0.57382131, 1.349748...",0.009921,11,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"[0.626637757, 0.26167056, -0.0240684357, 0.163...",0.017857,0,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"[0.415423244, -0.128793076, -0.201883331, 0.05...",0.020833,0,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,...,0.866667,0.527567,0.548553,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,"[0.35067722, 0.1657952, -0.11939768, 0.5296372...",0.027778,0,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2178,"[-1.67207658, 0.0626857281, -0.397092104, -3.6...",0.011905,11,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2179,"[0.346226931, 0.131596521, -0.390895218, 0.683...",0.008026,2,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2180,"[0.44693142, 0.00798898, -0.14514387, 1.489820...",0.206349,11,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,...,0.933333,0.664914,0.558473,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Split the 'prescription_rx_embeddings' column into separate columns based on array elements
expanded_columns = pd.DataFrame(df['prescription_rx_embeddings'].tolist(), index=df.index)

# Optionally, rename the columns to something meaningful
expanded_columns.columns = [f'P{i}' for i in range(expanded_columns.shape[1])]

# Concatenate the expanded columns back to the original DataFrame, if needed
df = pd.concat([df, expanded_columns], axis=1)

# Drop the original 'prescription_rx_embeddings' column
df.drop(columns=['prescription_rx_embeddings'], inplace=True)

df

,prescription_dose_val_rx,prescription_dose_unit_rx,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,pre_baseexcess,pre_totalco2,pre_hematocrit,pre_hemoglobin,...,P118,P119,P120,P121,P122,P123,P124,P125,P126,P127
0,0.010317,0,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,0.0,...,0.922822,0.194803,1.112494,-1.089337,-0.107142,-0.373226,-0.344020,0.328800,0.468490,-1.074432
1,0.011905,12,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,0.0,...,-0.004458,0.169775,-0.623910,0.348464,-0.027639,-0.209535,0.026674,0.787516,0.015712,-0.047965
2,0.009921,11,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,0.0,...,1.064011,-0.545910,1.657069,0.687448,0.182779,-0.156040,1.419005,1.244775,-0.725679,1.402903
3,0.017857,0,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,0.0,...,-0.486838,0.567405,-0.682332,0.124973,0.584291,0.036663,-0.033455,0.139260,0.164953,-0.279221
4,0.020833,0,0.500,0.741935,0.023018,0.0,0.545455,0.594595,0.0,0.0,...,-0.916522,0.936199,-0.727807,0.605511,0.442792,0.379367,0.042976,-0.075874,0.134937,0.000911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,0.027778,0,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,0.0,...,-0.844219,0.367455,-0.311357,-0.091761,0.238513,-1.155311,-1.137577,-0.644839,-0.607764,-0.205493
2178,0.011905,11,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,0.0,...,-0.449469,0.053109,-1.665534,-0.913456,0.804736,-1.414670,1.473890,1.771188,-2.091106,-1.964736
2179,0.008026,2,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,0.0,...,0.489259,-0.016742,0.511773,-0.010810,0.347862,0.296723,0.206720,0.449563,-0.123249,0.355982
2180,0.206349,11,0.375,0.500000,0.063939,0.0,0.303030,0.324324,0.0,0.0,...,0.143358,-0.577362,0.619785,0.227037,-0.856793,0.058450,-0.497501,-0.359619,-0.426077,0.646584


In [19]:
import boto3
import pandas as pd
from io import StringIO

# Define your bucket name and file key (file path in S3)
BUCKET_NAME = "blue-blood-data"
FILE_KEY = "BigQuery Blue Blood DB Data.csv"  # Change to your actual file path in S3

# Create an S3 client
s3 = boto3.client("s3")

# Fetch the file from S3
response = s3.get_object(Bucket=BUCKET_NAME, Key=FILE_KEY)

# Read the CSV file into a pandas DataFrame
csv_content = response["Body"].read().decode("utf-8")
df = pd.read_csv(StringIO(csv_content))

# Print DataFrame
print(df.head())

   subject_id   prescription_start     prescription_drug  \
0       10013  2125-10-05T00:00:00     Phenylephrine HCl   
1       10013  2125-10-05T00:00:00           Vasopressin   
2       10013  2125-10-05T00:00:00  Iso-Osmotic Dextrose   
3       10013  2125-10-05T00:00:00            Dobutamine   
4       10013  2125-10-05T00:00:00               Aspirin   

  prescription_dose_val_rx prescription_dose_unit_rx        pre_charttime  \
0                       60                        mg  2125-10-04T23:59:00   
1                      100                      UNIT  2125-10-04T23:59:00   
2                       50                        ml  2125-10-04T23:59:00   
3                      250                        mg  2125-10-04T23:59:00   
4                      325                        mg  2125-10-04T23:59:00   

   pre_ph  pre_pco2  pre_po2  pre_bicarbonate  ...  post_fio2_chartevents  \
0     7.3      63.0     60.0              NaN  ...              60.000002   
1     7.3      63.0   